In [52]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
import pandas_datareader.data as web


In [53]:
from dotenv import load_dotenv
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2') 

In [54]:
assets = api.list_assets()
tradeable = [asset for asset in assets if asset.tradable]

In [55]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

In [56]:
main_df = pd.DataFrame()
screened_stocks_df = pd.read_csv('../Data/REIT_stocks.csv')
screened_stocks_tickers = screened_stocks_df['Ticker']
tickers= screened_stocks_tickers.tolist()
tickers.pop(2)

'AIII'

In [57]:
timeframe = '1D'
end_date = datetime.now()
start_date = end_date + timedelta(-365)

for count, ticker in enumerate(tickers):
    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df
    df.index = df.index.date
    series = df[ticker]['close']
    series.rename(ticker, inplace =True)
    if main_df.empty:
        main_df = series
    else:
        main_df = pd.concat([main_df,series], axis = 'columns',join='inner')

        
main_df.head()

,UDR,UMH
2019-12-06,47.88,15.67
2019-12-09,47.84,15.75
2019-12-10,47.73,15.80
2019-12-11,46.80,16.20
2019-12-12,46.38,15.91


# Finding S&P 500 
## finding the close prices of sp500 with pandas DataReader

In [58]:
#THIS is the only thing that worked for me to get closing stock information
sp500 = web.DataReader(['sp500'], 'fred', start_date, end_date)
sp500.head()

,sp500
DATE,
2019-05-02,2917.52
2019-05-03,2945.64
2019-05-06,2932.47
2019-05-07,2884.05
2019-05-08,2879.42


In [59]:
read_stocks = web.DataReader(tickers, 'yahoo',start_date,end_date)
close_stocks = read_stocks['Close']
close_stocks.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-02,47.150002,15.410000,49.060001,24.010000,15.740000,201.440002,28.870001,11.10,14.00,99.980003,...,37.520000,26.299999,9.94,12.54,5.59,42.400002,123.839996,4.91,44.259998,13.89
2019-05-03,47.310001,16.030001,50.049999,24.280001,16.059999,202.160004,29.010000,11.29,14.05,100.830002,...,38.930000,28.410000,10.29,12.78,6.09,41.700001,123.709999,5.00,44.470001,13.95
2019-05-06,47.110001,16.420000,49.759998,24.160000,16.100000,200.570007,29.010000,11.40,14.19,100.889999,...,39.110001,28.030001,10.07,12.84,6.11,42.150002,123.919998,5.00,44.369999,13.65
2019-05-07,46.250000,16.160000,49.080002,23.660000,15.920000,196.789993,29.010000,11.11,14.18,99.070000,...,38.060001,26.889999,10.03,12.45,6.01,41.509998,121.839996,4.85,43.549999,13.15
2019-05-08,45.910000,16.100000,49.110001,23.530001,15.930000,196.449997,29.010000,11.04,14.22,98.750000,...,39.970001,26.809999,11.08,12.52,6.12,41.540001,121.150002,4.92,43.599998,13.22


In [60]:
sp500_returns = sp500.pct_change()
sp500_returns.dropna(inplace = True)
sp500_returns.head()

,sp500
DATE,
2019-05-03,0.009638
2019-05-06,-0.004471
2019-05-07,-0.016512
2019-05-08,-0.001605
2019-05-09,-0.003021


In [61]:
stocks_returns = close_stocks.pct_change()
stocks_returns.dropna(inplace =True)
stocks_returns.head()

Symbols,ACC,AHH,AIV,AMH,APTS,AVB,BOWFF,BRG,BRT,CPT,...,NXRT,OPI,RESI,RPT,SNR,SRC,SUI,UDFI,UDR,UMH
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-13,0.004533,0.012048,0.015524,0.014792,0.002320,0.012441,0.011032,0.006873,-0.009217,0.012620,...,0.017457,0.005979,-0.019964,0.007607,0.003995,0.006475,0.016620,-0.040000,0.014034,-0.013690
2019-11-14,0.005801,0.007937,0.003356,0.008439,0.029321,0.004868,0.000000,0.000000,0.005233,0.006593,...,0.007626,0.001564,0.002778,0.007550,0.002653,0.013671,0.004008,0.031250,0.007549,-0.014541
2019-11-15,0.005554,0.006187,0.007432,0.008368,0.020240,0.008983,0.057800,0.005119,-0.017930,0.007447,...,0.009249,0.000000,-0.000923,-0.007493,0.025132,0.006942,0.010137,-0.006734,0.008117,-0.004024
2019-11-18,0.008286,-0.010062,0.007008,-0.001886,0.005878,0.003449,0.016448,-0.008489,-0.004711,0.002672,...,0.001666,0.009994,0.001848,0.002745,0.015484,0.005318,0.012733,-0.003390,0.005161,-0.006734
2019-11-19,-0.000421,0.002823,0.001648,-0.002645,-0.005113,0.008315,-0.014262,0.000000,0.015976,0.001776,...,-0.006446,-0.005257,0.000000,0.001369,0.010165,0.002743,0.013688,0.020408,0.003902,0.010169


# Beta Section 

In [116]:
#covariance for the stocks with sp500
def get_covariance(stocks_returns):
    covariance_df = pd.DataFrame()
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        covariance_df = covariance_df.append(covariance)
    return covariance_dict


In [ ]:
#variance 
def get_variance():
    variance_dict = {}
    for symbol in stocks_returns:
        variance = stocks_returns[symbol].var()
        variance_dict.update({symbol: variance})
    return variance_dict


In [ ]:
#calculating beta
def get_beta(stocks_returns):
    beta_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        variance = stocks_returns[symbol].var()
        beta = covariance/variance
        beta_dict.update({beta:symbol})
    return beta_dict
beta_dict


# Rolling Averages 
## Getting a 20 day and 40 day rolling averages

In [ ]:
# Calculating Rolling Average of 20 days
rolling_20 = close_stocks.rolling(window = 20).mean()
rolling_20.dropna(inplace = True)
rolling_20.head()

In [ ]:
# Calculating Rolling Average of 40 days
rolling_40 = close_stocks.rolling(window = 40).mean()
rolling_40.dropna(inplace = True)
rolling_40.head()

In [ ]:
rolling_100 = close_stocks.rolling(window = 100).mean()
rolling_100.dropna(inplace =True)

# Top 5 for beta

In [ ]:
#getting the top pick of the day
beta_dict = get_beta(stocks_returns)  
search_key = 1
result = beta_dict.get(search_key) or beta_dict[min(beta_dict, key = lambda key: abs(key-search_key))]    
  
print("The stock best to choose today according to beta: " + str(result))


In [ ]:
#getting the top 5 picks of the day
def top_5_beta(beta_dict):
    sorted_beta_list = sorted(beta_dict, key=lambda key: abs(key), reverse = True)
    top_5_beta_list= sorted_beta_list[0:5]
    top_5_beta_dict = {}
    for keys in top_5_beta_list:
        for values in beta_dict.keys():
            if keys == values:
                top_5_beta_dict.update({beta[values]:keys})
    return top_5_beta_dict
top_5_beta_dict

# Filtered By Percent Change 
* within a 30 day range Can adust days if need be

In [113]:
avg_pct_change = stocks_returns.tail(30).mean()*100
avg_pct_change_df = pd.Series.to_frame(avg_pct_change)
avg_pct_change_df.columns = ['Percent_Change']
avg_pct_change_df.head()

,Percent_Change
Symbols,
ACC,1.674938
AHH,0.635270
AIV,0.834324
AMH,0.323655
APTS,0.624401


In [112]:
def top5_pct_change(pct_change_df):
    sorted_pct_df=pct_change_df.sort_values(by='Percent_Change', ascending=False)
    top5_df=sorted_pct_df.head()
    return top5_df

top5_pct_change(avg_pct_change_df)

,Percent_Change
Symbols,
SNR,2.428246
BOWFF,1.774461
ACC,1.674938
SRC,1.619407
GADS,1.383499


# Standard Deviation

In [123]:
def get_std(returns):
    std_dev_returns = returns.std()
    std_dev_df = pd.Series.to_frame(std_dev_returns)
    std_dev_df.columns = ['Std_Dev']
    return std_dev_df

get_std(stocks_returns)

,Std_Dev
Symbols,
ACC,0.051734
AHH,0.055504
AIV,0.045722
AMH,0.033751
APTS,0.049124
AVB,0.039036
BOWFF,0.059780
BRG,0.070727
BRT,0.051061


In [121]:
def top5_std_dev(std_dev_df):
    sorted_std_df=std_dev_df.sort_values(by='Std_Dev', ascending=False)
    top5_df=sorted_std_df.head()
    return top5_df
top5_std_dev(std_dev_df)

,Std_Dev
Symbols,
GADS,0.428367
SNR,0.072392
BRG,0.070727
RPT,0.059983
BOWFF,0.059780
